## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSBSBS'
meas='PRCP'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
2639
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|325.78201623339197|-315.8936352050001|59.289773260815096|      3.7|BSBSBSBS| 25.8175| -80.2858|       PRCP|0.9501597572800705|0.9082645853470678|0.9062953356908857|0.7597993003882532|USC00083909|308

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [10]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year     coeff_3
0  USC00083909  1982.0   59.289773
1  USW00012844  1959.0 -462.035098
2  USC00086315  2004.0  482.510751
3  USC00085653  1968.0 -428.399662
4  USW00092809  2003.0  242.079207

In [11]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  BF000078073  BFW00012712  BFW00012713  BFW00012716  US1FLBW0004  \
year                                                                       
1892.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   

station  US1FLBW0007  US1FLBW0009  US1FLBW0012  US1FLBW0013  US1FLBW0016  \
year                                                                       
1892.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USW00012852  USW00012859  USW00012882  USW00012885  \
year        ...                                                            
1892.0      ...               NaN          NaN          NaN          NaN   
1898.0      ...               NaN          NaN          NaN          NaN   
1899.0      ...               NaN          NaN          NaN          NaN   
1900.0      ...               NaN          NaN          NaN          NaN   
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   

station  USW00012888  USW00012895  USW00012896  USW00092805  USW00092809  \
year                                                                       
1892.0           NaN          NaN          NaN          NaN          NaN   
1898.0           NaN          NaN          NaN          NaN          NaN   
1899.0           NaN          NaN          NaN          NaN          NaN   
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   

station  USW00092811  
year                  
1892.0           NaN  
1898.0           NaN  
1899.0           NaN  
1900.0           NaN  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  

[10 rows x 152 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [12]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  190.175305112
RMS removing mean-by-station=  183.527239833
RMS removing mean-by-year   =  106.115119692


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 207.284761951
0 after removing mean by year    = 120.853088688
0 after removing mean by stations= 105.617399759
1 after removing mean by year    = 105.362906015
1 after removing mean by stations= 105.31573648
2 after removing mean by year    = 105.296013263
2 after removing mean by stations= 105.28532916
3 after removing mean by year    = 105.278826091
3 after removing mean by stations= 105.274621536
4 after removing mean by year    = 105.271810031
4 after removing mean by stations= 105.2698899
